# Find all the types of documents posible in supercias
### We need to define how many types documents and their frequency in supercias

In [6]:
packages = ['pymongo', 'gridfs', 'pandas']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        !pip install {package}

import os
from pymongo import MongoClient
import pandas as pd
#endpoint = '10.0.10.5:27017' # use vpn
endpoint = '192.168.1.10:27017'
#endpoint = '192.168.229.55:27017'
database = 'supercias'
collection = 'companies'
# Connect to MongoDB
db = MongoClient('mongodb://'+endpoint)[database]
# get collection
collection = db[collection]
#find pdf file
companies_cursor = collection.find({})
# get the first company
companies = list(companies_cursor)
print('Total companies:', len(companies))

Let seperate the docuemt by the type they can belong to: Generales, Economicos  and Judiciales

In [2]:
# now lets get all the documents for every company
companies = collection.find()
Documentos_generales = []
Documentos_economicos = []
Documentos_juridicos = []

for company in companies:
    # get the Documentos online
    print(company)
    #try:
    #    documentos = company['Documentos online']['downloaded']
    #    for documento in documentos['DocumentosGenerales']:
    #        Documentos_generales.append(documento)
    #    for documento in documentos['DocumentosEconomicos']:
    #        Documentos_economicos.append(documento)
    #    for documento in documentos['DocumentosJuridicos']:
    #        Documentos_juridicos.append(documento)
    #except KeyError:
     #   print(f'Could not find Documentos online for {company["name"]}, ruc: {company["ruc"]}')
    #    continue

ServerSelectionTimeoutError: 192.168.1.10:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 66f325ed9c65fc86cf392d2d, topology_type: Unknown, servers: [<ServerDescription ('192.168.1.10', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('192.168.1.10:27017: timed out (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>

Get all of the difrent kind of documentos generales into a df

In [4]:
docuemntos_generales_columns = ['ruc', 'tipo_doc', 'documento', 'fecha', 'nombre', 'cargo', 'other']
print(Documentos_generales)
docs = []
for documento in Documentos_generales:
    filename = documento
    fields = documento.split('.')[0].split('_')
    print(fields)
    #new_doc = {}
    #for index, field in enumerate(fields):
    #    print(index, field)
    #    if(index < len(docuemntos_generales_columns)):
    #        new_doc[docuemntos_generales_columns[index]] = field
    # add filename at the end
    #new_doc['filename'] = filename
    #docs.append(new_doc)
# make into a df 
#df_generales = pd.DataFrame(docs)
#df_generales.head()

Now get all of the the documentos economicos

In [4]:
documentos_economicos = []
for documento in Documentos_economicos:
    # remove the pdf extension
    documento = documento.replace('.pdf', '')
    # split by type
    new_doc = {}
    field = documento.split('_')
    new_doc['ruc'] = field[0]
    new_doc['tipo_doc'] = field[1]
    new_doc['documento'] = field[2]
    if len(field) > 3:
        new_doc['fecha'] = field[3]
    if len(field) > 4:
        new_doc['other'] = field[4]
    new_doc['filename'] = documento + '.pdf'
    documentos_economicos.append(new_doc)
# make a df with the documents
df_economicos = pd.DataFrame(documentos_economicos)
print(df_economicos.shape)
df_economicos.head()

(2441191, 5)


,ruc,tipo_doc,documento,fecha,filename
0,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1994-12-31,0991295437001_DocumentosEconomicos_Balance Es...
1,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1995-12-31,0991295437001_DocumentosEconomicos_Balance Es...
2,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1996-12-31,0991295437001_DocumentosEconomicos_Balance Es...
3,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1997-12-31,0991295437001_DocumentosEconomicos_Balance Es...
4,0991295437001,DocumentosEconomicos,Balance Estado de Situación Financiera,1998-12-31,0991295437001_DocumentosEconomicos_Balance Es...


now let make the documentos juridicos

In [5]:
documentos_juridicos = []
for documento in Documentos_juridicos:
    # remove the extension
    documento = documento.split('.')[0]
    # split by type
    new_doc = {}
    field = documento.split('_')
    new_doc['ruc'] = field[0]
    new_doc['tipo_doc'] = field[1]
    new_doc['acto_juridico'] = field[2]
    if len(field) > 3:
        new_doc['documento'] = field[3]
    if len(field) > 4:
        new_doc['fecha'] = field[4]
    if len(field) > 5:
        new_doc['other'] = field[7]
    new_doc['filename'] = documento + '.pdf'
    documentos_juridicos.append(new_doc)
# make a df with the documents
df_juridicos = pd.DataFrame(documentos_juridicos)
print(df_juridicos.shape)
df_juridicos.head()

(189181, 6)


,ruc,tipo_doc,acto_juridico,documento,fecha,filename
0,0991295437001,DocumentosJuridicos,CONSTITUCIÓN,Escritura,1994-09-01,0991295437001_DocumentosJuridicos_CONSTITUCIÓN...
1,0991295437001,DocumentosJuridicos,CONSTITUCIÓN,Extracto,1994-09-01,0991295437001_DocumentosJuridicos_CONSTITUCIÓN...
2,0991295437001,DocumentosJuridicos,AUMENTO DE CAPITAL,Escritura,2002-12-31,0991295437001_DocumentosJuridicos_AUMENTO DE C...
3,0991295437001,DocumentosJuridicos,AUMENTO DE CAPITAL,Extracto,2002-12-31,0991295437001_DocumentosJuridicos_AUMENTO DE C...
4,0991295437001,DocumentosJuridicos,CAMBIO DE OBJETO SOCIAL,Escritura,2006-02-16,0991295437001_DocumentosJuridicos_CAMBIO DE OB...


Check the pdfs for the pdfs that we don't have the complete title or filename for
this it important because based on the document name we define how we can extract the information 

In [14]:
# get all the documents generales with no docuemnto column
df_generales_nan = df_generales[df_generales['fecha'].isna()]
df_generales.head()
#df_generales = df_generales[~df_generales['fecha'].isna()]
#print(f'In the df_generales there are {df_generales_nan.shape[0]} out of {df_generales.shape[0]} with no fecha column')

#df_generales['documento'].value_counts()

,ruc,tipo_doc,documento,fecha,filename,nombre,cargo
0,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,1994-09-13,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
1,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,1996-09-06,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
2,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2008-05-08,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
3,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2009-01-14,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN
4,0991295437001,DocumentosGenerales,Oficio Transferencia Acciones,2013-05-13,0991295437001_DocumentosGenerales_Oficio Trans...,NaN,NaN


In [7]:

# make the same for economicos
#df_economicos['documento'].value_counts().shape

In [8]:
df_juridicos_fecha_nan = df_juridicos[df_juridicos['fecha'].isna()]

df_juridicos_fecha_nan.shape

# get all docuemntos de 2023
#df_juridicos[df_juridicos['fecha'].str.contains('2023')].head()
#df_juridicos['acto_juridico'].value_counts().shape
# make the same for juridicos
#df_juridico_2023 = df_juridicos['acto_juridico'].value_counts()


(2505, 6)